# 1.读取CSV文件里的数据

这里要注意一点，如果你的代码以来了自己自定义的莫快，需要使用sys.path.append添加
项目根路径，不然的话，python执行会找不到模块，而且必须写在import之前。

In [2]:
import sys
import os
import pandas as pd
import numpy as np
sys.path.append(os.path.abspath(os.path.join(os.getcwd(),"../../")))

result_save_file = 'wzly.csv'
raw_data = pd.read_csv(result_save_file)

# 2.分析身高情况

这里要对数据坐下清洗，流程如下：

- （1）筛选140-200以内的身高
- （2）将身高按照升序排列
- （3）按照三个身高段，对数据进行分段处理，依次是140，150，160，170，180

In [11]:
from pyecharts import Bar

# 身高范围
height_interval = ['140', '150', '160', '170', '180']  

# 分析身高
def analysis_height(data):
    height_data = data['身高']
    height = (height_data.loc[(height_data > 140) & (height_data < 200)]).value_counts().sort_index()
    height_count = [0, 0, 0, 0, 0]
    for h in range(0, len(height)):
        if 140 <= height.index[h] < 150:
            height_count[0] += height.values[h]
        elif 150 <= height.index[h] < 160:
            height_count[1] += height.values[h]
        elif 160 <= height.index[h] < 170:
            height_count[2] += height.values[h]
        elif 170 <= height.index[h] < 180:
            height_count[3] += height.values[h]
        elif 180 <= height.index[h] < 190:
            height_count[4] += height.values[h]
    return height_count

# 绘制身高分布柱状图
def draw_height_bar(data):
    bar = Bar("妹子身高分布柱状图")
    bar.add("妹子身高", height_interval, data, bar_category_gap=0, is_random=True, )
    return bar

draw_height_bar(analysis_height(raw_data))

In [18]:
from pyecharts import Pie

# 绘制身高分布饼图
def draw_height_pie(data):
    pie = Pie("妹子身高分布饼图-圆环图", title_pos='center')
    pie.add("", height_interval, data, radius=[40, 75], label_text_color=None,
            is_label_show=True, legend_orient='vertical', is_random=True,
            legend_pos='left')
    return pie

draw_height_pie(analysis_height(raw_data))

# 3.分析学历

接着分析一下学历的情况，这里我们用到漏斗图

In [23]:
from pyecharts import Funnel

# 学历范围
edu_interval = ['本科', '大专', '高中', '中专', '初中', '硕士', '博士', '院士']  

# 分析学历
def analysis_edu(data):
    return data['学历'].value_counts()

# 学历漏斗图
def draw_edu_funnel(data):
    funnel = Funnel("妹子学历分布漏斗图",title_top='center')
    funnel.add("学历", edu_interval, data, is_label_show=True,
               label_pos="inside", label_text_color="#fff", is_random=True)
    return funnel

draw_edu_funnel(analysis_edu(raw_data))

# 4.分析年龄情况

接下来分析下都是些什么年龄段的妹子在找对象，这里还是先要过滤一些不合理的数据，
筛选的合理年龄段范围为：1956-2000，分为五个年龄段：18-25，25-30，31-40，41-50和50以上。

In [47]:
from pyecharts import Radar

# 年龄范围
age_interval = [
    ('18-25', 8000), ('26-30', 8000), ('31-40', 8000),
    ('41-50', 8000), ('50以上', 8000),
]

# 分析年龄
def analysis_age(data):
    age_data = data['出生年份']
    age = (age_data.loc[(age_data >= 1956) & (age_data <= 2000)]).value_counts().sort_index()
    age_count = [[0, 0, 0, 0, 0]]
    for h in range(0, len(age)):
        if 1993 <= age.index[h] <= 2000:
            age_count[0][0] += age.values[h]
        elif 1988 <= age.index[h] <= 1992:
            age_count[0][1] += age.values[h]
        elif 1978 <= age.index[h] <= 1987:
            age_count[0][2] += age.values[h]
        elif 1968 <= age.index[h] <= 1977:
            age_count[0][3] += age.values[h]
        elif age.index[h] < 1968:
            age_count[0][4] += age.values[h]
    return age_count

# 年龄雷达图
def draw_age_radar(data):
    radar = Radar("妹子年龄分布雷达图")
    radar.config(age_interval)
    radar.add("年龄段", data, is_area_show=False, is_splitline_show='single')
    return radar

draw_age_radar(analysis_age(raw_data))

# 5.分析城市分布

接着我们来分析下找对象妹子的城市分布，这里用到了地图

In [4]:
from pyecharts import Geo

# 分析城市分布
def analysis_city(data):
    city_data = data['城市'].value_counts()
    city_list = []
    for city in range(0, len(city_data)):
        if city_data.values[city] > 10 and city_data.index[city] != '国外' :
            city_list.append((city_data.index[city], city_data.values[city]))
    return city_list

# 城市分布地图
def draw_city_geo(data):
    geo = Geo("全国妹子分布城市", "制作人：CoderPig", title_color="#fff",
              title_pos="left", width=1200,
              height=600, background_color='#404a59')
    attr, value = geo.cast(data)
    geo.add("", attr, value, visual_range=[10, 2500], visual_text_color="#fff",
            symbol_size=15, is_visualmap=True)
    return geo

draw_city_geo(analysis_city(raw_data))

# 6.分析交友宣言

接着我们来分析下妹子们的交友宣言，看看妹子们都喜欢怎么样的男生，需要过滤一下网站默认的交友宣言，一般自己写的都是不会重复的，这个可以利用pandas.value_counts()，只保留出现一次的交友宣言，然后利用Jieba分词，Counter统计词频，剔除标点和特殊符号，同时过滤掉一些无意义的无用词，然后取频次最高的100个关键词，绘制成词云，以此了解妹子们的喜好。具体代码如下：

In [13]:
from pyecharts import WordCloud
import re
import jieba as jb
from collections import Counter

# 过滤标点的正则
word_pattern = re.compile('[\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？“”、~@#￥%……&*（）(\d+)]+')
# 过滤无用词
exclude_words = [
            '一辈子', '不相离', '另一半', '业余时间', '性格特点', '茫茫人海', '男朋友', '找对象',
            '谈恋爱', '有时候', '女孩子', '哈哈哈', '加微信', '兴趣爱好',
            '是因为', '不良嗜好', '男孩子', '为什么', '没关系', '不介意',
            '没什么', '交朋友', '大大咧咧', '大富大贵', '联系方式', '打招呼',
            '有意者', '晚一点', '哈哈哈', '以上学历', '是不是', '给我发',
            '不怎么', '第一次', '越来越', '遇一人', '择一人', '无数次',
            '符合条件', '什么样', '全世界', '比较简单', '浪费时间', '不知不觉',
            '有没有', '寻寻觅觅', '自我介绍', '请勿打扰', '差不多', '不在乎', '看起来',
            '一点点', '陪你到', '这么久', '看清楚', '身高体重', '比较慢', '比较忙',
            '多一点', '小女生', '土生土长', '发消息', '最合适','有缘人','过日子'
        ]

# 词频分布
def analysis_word(data):
    word_data = data['交友宣言'].value_counts()
    word_list = []
    for word in range(0, len(word_data)):
        if word_data.values[word] == 1:
            word_list.append(word_data.index[word])
    return word_list


# 绘制交友宣言词云
def draw_word_wc(name, count):
    wc = WordCloud(width=900, height=720)
    wc.add("", name, count, word_size_range=[20, 100], shape='circle')
    return wc
    
word_result = word_pattern.sub("", ''.join(analysis_word(raw_data)))
words = [word for word in jb.cut(word_result, cut_all=False) if len(word) >= 3]
# 遍历过滤掉无用词
for i in range(0, len(words)):
    if words[i] in exclude_words:
        words[i] = None
filter_list = list(filter(lambda t: t is not None, words))
data = r' '.join(filter_list)
# 词频统计
c = Counter(filter_list)
word_name = []  # 词
word_count = []  # 词频
for word_freq in c.most_common(100):
    word, freq = word_freq
    word_name.append(word)
    word_count.append(freq)
    
draw_word_wc(word_name, word_count)